<a href="https://colab.research.google.com/github/sujithrc/Fine-Tunning-using-LORA/blob/main/Fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers peft datasets accelerate



INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.5 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig
from datasets import load_dataset, Dataset



In [4]:
model_name = "distilbert-base-uncased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Set the model to use GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [5]:
context = "Hugging Face is creating a tool that democratizes AI."
question = "What is Hugging Face creating?"


In [6]:
def get_answer(model, context, question):
    inputs = tokenizer(question, context, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        answer_start = outputs.start_logits.argmax()
        answer_end = outputs.end_logits.argmax() + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer


In [7]:
print("Answer before fine-tuning:")
answer_before = get_answer(model, context, question)
print(answer_before)


Answer before fine-tuning:
a tool that democratizes ai


In [29]:
# Print the model architecture
print(model)


DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [8]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=2,
    logging_dir='./logs',
    logging_steps=10,
    remove_unused_columns=True,
)


In [9]:
data = {
    "context": ["This is a sample context."],
    "question": ["What is this?"],
    "answers": [{"text": ["a sample context"], "answer_start": [10]}]
}

train_dataset = Dataset.from_dict(data)




In [10]:
def preprocess_function(examples):
    questions = examples["question"]
    contexts = examples["context"]
    inputs = tokenizer(questions, contexts, padding="max_length", truncation=True, max_length=512)

    start_positions = []
    end_positions = []

    for i in range(len(examples["answers"])):
        answer = examples["answers"][i]["text"][0]  # Use the first answer
        context = contexts[i]

        start = context.find(answer)
        end = start + len(answer) if start != -1 else -1

        # Validate positions
        if start != -1 and end != -1:
            start_positions.append(start)
            end_positions.append(end)
        else:
            start_positions.append(-1)
            end_positions.append(-1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'start_positions': start_positions,
        'end_positions': end_positions
    }



In [11]:
train_dataset = train_dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)


In [34]:
print(train_dataset)


Dataset({
    features: ['context', 'question', 'answers'],
    num_rows: 1
})


In [13]:
trainer.train()
lora_model.save_pretrained("./lora_finetuned_model")




Step,Training Loss


TrainOutput(global_step=2, training_loss=5.578247547149658, metrics={'train_runtime': 16.8847, 'train_samples_per_second': 0.118, 'train_steps_per_second': 0.118, 'total_flos': 261306200064.0, 'train_loss': 5.578247547149658, 'epoch': 2.0})

In [15]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")



('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

NOW WE ARE IN THE "FINAL-PART"  WE FINE-TUNNED(32bit into 4bit) OUR MODEL.

 """ LETS SEE HOW ITS WORKING FOR THE SAME SAMPLE DATA""""

In [17]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Load the model and tokenizer
model_name = "distilbert-base-uncased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the fine-tuned model for quantization
quantized_model = AutoModelForQuestionAnswering.from_pretrained(
    "/content/fine_tuned_model", torch_dtype=torch.float16
)

# Set the model to use GPU (if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
quantized_model.to(device)

# Define the context and question
context = "Hugging Face is creating a tool that democratizes AI."
question = "What is Hugging Face creating?"

# Function to get answer from the model
def get_answer(model, context, question):
    inputs = tokenizer(question, context, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        answer_start = outputs.start_logits.argmax()
        answer_end = outputs.end_logits.argmax() + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer

# Get the answer after fine-tuning
print("\nAnswer after fine-tuning:")
answer_after = get_answer(quantized_model, context, question)
print(answer_after)




Answer after fine-tuning:
a tool that democratizes ai
